In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from google.colab import drive
import os
drive.mount('/content/drive/', force_remount = True)
ROOT_PATH="/content/drive/My Drive/Udem/Winter2021/IFT 6756 - Game Theory and ML/Term Project/disp_bert_defender"
os.chdir(ROOT_PATH)
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at /content/drive/


'/content/drive/My Drive/Udem/Winter2021/IFT 6756 - Game Theory and ML/Term Project/disp_bert_defender'

In [ ]:
!pip install torch
!which python
!pip install boto3
!pip install hnswlib==0.5.1
!pip install sklearn
!pip install pandas
!pip install nltk
import nltk
nltk.download('punkt')




/usr/local/bin/python
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
num_disc_epochs = 5
num_gen_epochs = 5
num_disc_epochs_eval = num_disc_epochs - 1 
num_gen_epochs_eval = num_gen_epochs - 1 
disc_test_file = "data/sst-2/disc/disc_enum_swap_base.tsv"
disc_test_file = "data/sst-2/disc/disc_enum_rand_base.tsv"
disc_test_file = "data/sst-2/disc/disc_enum_drop_base.tsv"
# disc_test_file = "data/sst-2/disc/disc_enum_add_base.tsv"
# class_test_file = "data/sst-2/disc/disc_enum_add_base.tsv"
folder  = "./data/sst-2/"
folder_disc  = "./data/sst-2/disc"
folder_gen  = "./data/sst-2/gen"
# folder_disc = folder_gen = folder
word_embedding_file = "./emb/wiki-news-300d-1M-s.vec"

In [ ]:
!pip install urllib3
!pip install chardet

!pip3 install requests
!pip3 install folium 

In [ ]:
# !nvidia-smi
# !nvidia-smi -q

In [ ]:
import torch
torch.cuda.is_available()
import tensorflow as tf
tf.test.gpu_device_name()

True

'/device:GPU:0'

# Testing

Inference
We first attack the test data using 5 differernt methods to drop the model performance as much as possible. The codes related to attacking the test sets would be availble soon!

During inference phase, we use the pre-trained discriminator to identify the words that have been attacked.

In [1]:
# Modified - Run this : 
# Discriminator do_eval or Testing 
!python bert_discriminator.py \
--task_name sst-2 \
--do_eval \
--eval_batch_size 32 \
--do_lower_case \
--data_dir $folder_disc \
--data_file $disc_test_file \
--bert_model bert-base-uncased \
--max_seq_length 128 \
--train_batch_size 16 \
--learning_rate 2e-5 \
--output_dir $folder_disc \
--num_eval_epochs $num_disc_epochs_eval \
--word_embedding_file $word_embedding_file \
--single  \
--verbose
# --no_cuda 

python3: can't open file 'bert_discriminator.py': [Errno 2] No such file or directory


Then, we recover the words with a pre-trained embedding estimator. Note that we use small-world-graph to conduct a KNN-based search for closest word in the embedding space.

In [2]:
!python bert_generator.py \
--task_name sst-2 \
--do_eval \
--do_lower_case \
--data_dir $folder_disc \
--data_file $disc_test_file \
--bert_model bert-base-uncased \
--max_seq_length 64 \
--train_batch_size 8 \
--learning_rate 2e-5 \
--output_dir $folder_gen \
--word_embedding_file $word_embedding_file \
--single  \
--num_eval_epochs $num_gen_epochs_eval
# --no_cuda 

python3: can't open file 'bert_generator.py': [Errno 2] No such file or directory


# Classification

After recovering the test instances, we can run a model to check the recovering effectiveness. The model in our settings is a sentiment classification model based on bert contextualized embeddings.

In [ ]:
!python bert_classifier.py \
--task_name sst-2 \
--num_train_epochs 5 \
--do_eval \
--do_lower_case \
--data_dir $folder_gen \
--bert_model bert-base-uncased \
--max_seq_length 64 \
--train_batch_size 8 \
--learning_rate 2e-5 \
--output_dir data/sst-2/classifier \
--no_cuda 

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
04/26/2021 23:43:36 - INFO - bert_utils -   device: cpu n_gpu: 1, distributed training: False, 16-bits training: False
04/26/2021 23:43:36 - INFO - tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/26/2021 23:43:37 - INFO - bert_utils -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/26/2021 23:43:37 - INFO - bert_utils -   extracting archive file /root/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463